# Fast Fourier Transform

## 12.01 Discrete Fourier Transform

Represent a continous function as linear combination of sines and cosines.
* The sine and cosine components are referred to as **frequencies**.
* For some problems, frequency domain is more efficient than time or space domain.

**Euler's formula**
$$
e^{i \theta} = \cos \theta + i \sin \theta
$$
where
* $i = \sqrt{-1}$ is complex

**Roots of Unity**

aka **twiddle factors**: For a given integer $n$ the primitive nth root of unity is given by:
$$
\omega_n = \cos(2\pi/n) - i \sin(2\pi/n) = e^{-2\pi i/n}
$$

#### Discrete Fourier Transform
DFT $y$ of the sequence $x = [x_0, \cdots, x_{n-1}]^T$ is given by:
$$
y_m = \sum_{k=0}^{n-1} {x_k \omega_n^{mk}}, \qquad m = 0, \cdots, n-1
$$

Expressed in matrix notation:
$$
y = F_n x
$$
where
* $F_n$ is the Fourier matrix with entries $F_n = \omega_n^{mk}$.

#### Inverse DFT
Inverse DFT $x$ of the sequence $y = [y_0, \cdots, y_{n-1}]^T$ is given by:
$$
x_k = \frac{1}{n} \sum_{m=0}^{n-1} {y_m \omega_n^{-mk}}, \qquad k = 0, \cdots, n-1
$$

Expressed in matrix notation:
$$
F_n^{-1} = (1/n) F_n^H
$$

## 12.02 Fast Fourier Transform

## 12.03 Applications of DFT

## 12.04 Wavelets